In [21]:
import pandas as pd

from mlinspect.utils import get_project_root

patients = pd.read_csv("{}/example_pipelines/healthcare/patients.csv".format(get_project_root()),
                       na_values='?')
histories = pd.read_csv("{}/example_pipelines/healthcare/histories.csv".format(
    get_project_root()), na_values='?')

merged = histories.merge(patients, on=['ssn'])
complications = merged.groupby('age_group') \
    .agg(mean_complications=('complications', 'mean'))
merged = merged.merge(complications, on=['age_group'])
merged['label'] = merged['complications'] > 1.2 * merged['mean_complications']

# `smoker`

In [22]:
import random
mask = random.choices([0, 1], weights=[0.4, 0.6], k=merged.shape[0])

In [23]:
mask

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,


In [24]:
# merged['new_smoker'] = merged['smoker'].where(merged['smoker'].isna(), merged['label'] * mask)

mask_bool = (mask & merged['smoker'].notna()).to_numpy()
merged['mask_bool'] = mask_bool
merged['new_smoker'] = merged['smoker']
merged.loc[mask_bool, 'new_smoker'] = merged['label']

In [25]:
mask_bool

array([False, False,  True,  True, False, False, False,  True, False,
        True,  True,  True,  True,  True, False, False,  True,  True,
       False, False,  True, False, False, False,  True,  True, False,
       False, False,  True,  True, False, False,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
       False, False,  True,  True, False,  True, False, False, False,
        True,  True,  True,  True, False, False, False,  True, False,
        True, False, False, False, False,  True, False,  True, False,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
       False, False, False,  True, False,  True,  True, False, False,
       False,  True,  True,  True,  True, False,  True, False,  True,
       False,  True, False, False,  True, False,  True,  True,  True,
        True, False,

In [26]:
merged[['smoker', 'label', 'new_smoker', 'mask_bool']].head(15)

,smoker,label,new_smoker,mask_bool
0,False,False,False,False
1,False,False,False,False
2,False,False,False,True
3,True,True,True,True
4,False,False,False,False
5,False,False,False,False
6,True,True,True,False
7,False,False,False,True
8,NaN,True,NaN,False
9,True,True,True,True


In [27]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 887 entries, 0 to 886
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   smoker              817 non-null    object 
 1   complications       887 non-null    int64  
 2   ssn                 887 non-null    object 
 3   id                  887 non-null    int64  
 4   first_name          887 non-null    object 
 5   last_name           887 non-null    object 
 6   race                792 non-null    object 
 7   county              704 non-null    object 
 8   num_children        887 non-null    int64  
 9   income              887 non-null    float64
 10  age_group           887 non-null    object 
 11  mean_complications  887 non-null    float64
 12  label               887 non-null    bool   
 13  mask_bool           887 non-null    bool   
 14  new_smoker          817 non-null    object 
dtypes: bool(2), float64(2), int64(3), object(8)
memory usage:

In [28]:
merged['smoker'] = merged['new_smoker']
merged[['smoker','complications','ssn']].to_csv("{}/example_pipelines/healthcare/histories.csv".format(
    get_project_root()), index=False)

# `income`

In [8]:
merged['new_income'] = merged['income']
merged.loc[merged['label'].to_numpy(), 'new_income'] = merged['income'] * 1.5
merged.loc[~merged['label'].to_numpy(), 'new_income'] = merged['income'] * 0.75

In [9]:
merged[['income', 'new_income', 'label']].head(15)

,income,new_income,label
0,212036,159027.00,False
1,11824,8868.00,False
2,160229,120171.75,False
3,146742,220113.00,True
4,117328,87996.00,False
5,162280,121710.00,False
6,234791,352186.50,True
7,16039,12029.25,False
8,222278,333417.00,True
9,161243,241864.50,True


In [10]:
merged['income'] = merged['new_income']

In [11]:
merged[['id', 'first_name','last_name','race','county','num_children','income','age_group','ssn']]\
    .to_csv("{}/example_pipelines/healthcare/patients.csv".format(
    get_project_root()), index=False)

# Healthcare pipeline

In [29]:
"""Predicting which patients are at a higher risk of complications"""
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from example_pipelines.healthcare.healthcare_utils import MyW2VTransformer, MyKerasClassifier, \
    create_model
from mlinspect.utils import get_project_root

patients = pd.read_csv("{}/example_pipelines/healthcare/patients.csv".format(get_project_root()),
                       na_values='?')
histories = pd.read_csv("{}/example_pipelines/healthcare/histories.csv".format(
    get_project_root()), na_values='?')

data = patients.merge(histories, on=['ssn'])
complications = data.groupby('age_group') \
    .agg(mean_complications=('complications', 'mean'))
data = data.merge(complications, on=['age_group'])
data['label'] = data['complications'] > 1.2 * data['mean_complications']
# data = data[(data['age_group'] == 'group1') | (data['age_group'] == 'group2')]
data = data[['smoker', 'last_name', 'county', 'num_children', 'race', 'income', 'label']]
data = data[data['county'].isin(['county2', 'county3'])]  # Add or remove this line, modify counties
# data = data[data['race'] == 'race3']

# data = data.dropna()  # Add this line and/or remove the imputer
impute_and_one_hot_encode = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder(sparse=False, handle_unknown='ignore'))  # Test sparse=True
])
featurisation = ColumnTransformer(transformers=[
    ("impute_and_one_hot_encode", impute_and_one_hot_encode,
     ['smoker', 'county', 'race']),  # Remove/add feature columns, e.g., remove race
    ('word2vec', MyW2VTransformer(min_count=2), ['last_name']),
    ('numeric', StandardScaler(), ['num_children', 'income'])
])
neural_net = MyKerasClassifier(build_fn=create_model, epochs=10, batch_size=1, verbose=0)
pipeline = Pipeline([
    ('features', featurisation),
    ('learner', neural_net)])

train_data, test_data = train_test_split(data)
model = pipeline.fit(train_data, train_data['label'])  # Use/do not use a train/test split
print("Mean accuracy: {}".format(model.score(test_data, test_data['label'])))

Mean accuracy: 0.931506872177124
